# Route planner

> Module die alle functies bevat die nodig zijn om de snelste route te vinden langs een gegeven serie locaties.
> Deze maakt gebruik van de API van [openrouteservice](https://openrouteservice.org/dev/#/api-docs).
> Deze functies hoeven in principe niet door de gebruiker zelf te worden aangeroepen.

In [ ]:
#| default_exp route_get

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from datetime import datetime
import os

import pandas as pd
import numpy as np
from fastcore.utils import L

import openrouteservice
from geopy.geocoders import Nominatim

from project.data_get import get_data_from_azuresql
from project.utils import load_settings

In [ ]:
#| export
# only used for testing while developping
settings = load_settings()
settings

{'files': {'path_peilbuizenshape_file': '/home/jelle/code/peilbuizen_route_optimalisatie/data/Grondwatermeetpunten/Grondwatermeetpunten_2024-10-10.shp',
  'path_pickle_results': '/home/jelle/code/peilbuizen_route_optimalisatie/data/pickles',
  'save_to_pickle': True,
  'pickle_file_input': '/home/jelle/code/peilbuizen_route_optimalisatie/data/pickles/get_data_from_azuresql_20250124_130111.pickle',
  'path_results': '/home/jelle/results/peilbuizen_route_optimalisatie'},
 'filters': {'days_since_last_update': 200, 'projects': 'primair'},
 'calculation': {'distance_calculation_method': 'cycling-regular',
  'startlocation': 'Dokter van Deenweg 186, 8025 BM, Zwolle'},
 'azure': {'jdbc_hostname': 'wdodeuwprodsynoxgn02-ondemand.sql.azuresynapse.net',
  'jdbc_database': 'Lakehouse'},
 'sql_statement': {'tsid': "select [Site_Name], [Site_Longname], [Site_Number], SI.[STAS_ID], STA.[STA_ID], [Station_name], [Timeseries_name], [TS_ID], [WARECO_id]from [Datamart].[DIM_WDOD_STATIONS] ST, [Datamart]

In [ ]:
#| export
# Used to test
from fastcore.utils import Path
from project.data_get import load_pickle

file_path = settings['files']['path_pickle_results']
file_name = "get_data_from_azuresql_20250124_130111.pickle"
peilbuizen_df = load_pickle(file_path=Path(file_path) / file_name)

df_grouped = peilbuizen_df.groupby('project')
test_df = df_grouped.get_group('---')

In [ ]:
#| export

client = openrouteservice.Client(key=os.environ["OPENROUTESERVICE_KEY"])

Get the coordinates from the start location and from the travel locations.

In [ ]:
#| export
def get_lonlat_start_location(address: str = "Dokter van Deenweg 186, 8025 BM, Zwolle"):
    """Get the longitude and latitude coordinates from a given adress"""
    geolocator = Nominatim(user_agent="wdodelta_route_optimizer", timeout=10)
    max_retries = 3
    retry_delay = 2

    for attempt in range(max_retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return (location.longitude, location.latitude)
            else:
                print(f"Warning: Could not find coordinates for adress: {address}")
                return None
        except GeocoderTimedOut:
            if attempt < max_retries - 1:
                print(f"Warning: Geocoder timed out, attempt {attempt + 1}/{max_retries}. Retrying...")
                time.sleep(retry_delay)
            else:
                print(f"Error: Geocoder timed out after {max_retries} attempts")
                return None
        except GeocoderServiceError as e:
            print(f"Error: Geocoding service error - {str(e)}")
            return None
        except Exception as e:
            print(f"Error: Unexpected error during geocoding - {str(e)}")
            return None

def df_to_longlat_tuple(df, longitude_column, latitude_column):
    """Get the longitude and latitude coordinates from all rows in a pandas dataframe
    and return a list of longitude, latitude tuples"""
    return [(row[longitude_column], row[latitude_column]) for _, row in df.iterrows()]


In [ ]:
#| export

start_location = get_lonlat_start_location(settings['calculation']['startlocation'])
start_location

(6.1254037, 52.5069559)

In [ ]:
#| export

longlat_tpl = df_to_longlat_tuple(test_df, longitude_column="Longitude", latitude_column="latitude")
longlat_tpl

[(6.1806743495199, 52.359436355642),
 (6.20297998352629, 52.30628223108),
 (6.20765256006221, 52.6018140077626),
 (6.18105510555902, 52.6139544769308),
 (6.21955057665591, 52.6502456191275),
 (6.20970452646494, 52.513289835589),
 (6.22363135268854, 52.5983052709003),
 (6.34958727039631, 52.50067890897),
 (6.32240426396448, 52.4994853087927)]

Map the travel locations order to the Peilbuizen from the peilbuizen dataframe.

This has to be done using the proximity of the points, because the returned coordinates of the route optimalisation can be shifted by some distance with respect to the input coordinates.

Find the shortest route through all locations starting at the start location and ending at the start location. Using the openrouteservice API.

In [ ]:
#| export
def create_optimized_route(start_address: str,
                           df: pd.DataFrame,
                           route_profile: str,
                           long_clmn: str="Longitude",
                           lat_clmn: str="latitude"):
    """Solve the traveling salesman problem (visit all given points exactly once in the 
    shortes possible route) for a given start adress and pandas dataframe with longitude
    and latitude columns."""
    start_coords = get_lonlat_start_location(start_address)
    peilbuizen_coords = df_to_longlat_tuple(df, longitude_column=long_clmn, latitude_column=lat_clmn)
    total_coords = L([start_coords] + peilbuizen_coords + [start_coords])
    return client.directions(total_coords,
                           profile=route_profile,
                           optimize_waypoints=True,
                           instructions=False,
                           geometry=True,
                           format='geojson')


In [ ]:
#| export

optimal_route = create_optimized_route(start_address=settings['calculation']['startlocation'],
                                       route_profile=settings['calculation']['distance_calculation_method'],
                                       df=test_df)
optimal_route

{'type': 'FeatureCollection',
 'features': [{'bbox': [4.41667, 51.226897, 6.350065, 52.650295],
   'type': 'Feature',
   'properties': {'way_points': [0,
     3737,
     4167,
     4185,
     4559,
     4785,
     4845,
     4852,
     4989,
     5324,
     9013],
    'summary': {'distance': 517218.6, 'duration': 106922.79999999999}},
   'geometry': {'coordinates': [[4.41667, 51.233656],
     [4.416674, 51.232149],
     [4.417421, 51.232153],
     [4.417424, 51.232106],
     [4.417425, 51.232051],
     [4.417225, 51.23205],
     [4.417227, 51.231827],
     [4.417227, 51.231775],
     [4.418174, 51.231775],
     [4.418785, 51.231777],
     [4.41931, 51.231775],
     [4.41947, 51.231788],
     [4.419681, 51.231833],
     [4.419893, 51.231933],
     [4.42005, 51.232075],
     [4.420096, 51.232163],
     [4.420127, 51.232817],
     [4.420125, 51.234415],
     [4.42012, 51.234654],
     [4.42007, 51.234896],
     [4.420068, 51.234933],
     [4.420247, 51.234935],
     [4.420601, 51.234935],

Extract the order in which the travel locations are visited in the optimized route.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()